In [1]:
import numpy as np
import xarray as xr
from osprey.actions.reader import reader_nemo
from osprey.means.means import elements

data = reader_nemo(expname='lfr0', startyear=1990, endyear=2010)
df = elements('ORCA2')
ave = data['thetao'].weighted(df['V']).mean(dim=['z', 'y', 'x'])
ave_result= ave.compute()

2024-09-26 13:54:10,508 - INFO - Files to be loaded ['/ec/res4/scratch/itas/ece4/lfr0/output/nemo/lfr0_oce_1m_T_1990-1990.nc', '/ec/res4/scratch/itas/ece4/lfr0/output/nemo/lfr0_oce_1m_T_1991-1991.nc', '/ec/res4/scratch/itas/ece4/lfr0/output/nemo/lfr0_oce_1m_T_1992-1992.nc', '/ec/res4/scratch/itas/ece4/lfr0/output/nemo/lfr0_oce_1m_T_1993-1993.nc', '/ec/res4/scratch/itas/ece4/lfr0/output/nemo/lfr0_oce_1m_T_1994-1994.nc', '/ec/res4/scratch/itas/ece4/lfr0/output/nemo/lfr0_oce_1m_T_1995-1995.nc', '/ec/res4/scratch/itas/ece4/lfr0/output/nemo/lfr0_oce_1m_T_1996-1996.nc', '/ec/res4/scratch/itas/ece4/lfr0/output/nemo/lfr0_oce_1m_T_1997-1997.nc', '/ec/res4/scratch/itas/ece4/lfr0/output/nemo/lfr0_oce_1m_T_1998-1998.nc', '/ec/res4/scratch/itas/ece4/lfr0/output/nemo/lfr0_oce_1m_T_1999-1999.nc', '/ec/res4/scratch/itas/ece4/lfr0/output/nemo/lfr0_oce_1m_T_2000-2000.nc', '/ec/res4/scratch/itas/ece4/lfr0/output/nemo/lfr0_oce_1m_T_2001-2001.nc', '/ec/res4/scratch/itas/ece4/lfr0/output/nemo/lfr0_oce_1m_T_

```
import numpy as np
import xarray as xr
import logging
from osprey.actions.reader import reader_nemo
from osprey.means.means import elements
from dask.distributed import LocalCluster, Client, progress
from dask.distributed.diagnostics import MemorySampler

data = reader_nemo(expname='lfr0', startyear=1990, endyear=2010)

cluster = LocalCluster(n_workers=2, threads_per_worker=1, memory_limit='8GB')
client = Client(cluster)
df = elements('ORCA2')
ave = data['thetao'].weighted(df['V']).mean(dim=['z', 'y', 'x'])
ms = MemorySampler()
with ms.sample('chunk'):
    ave_result= ave.compute()
array_data = np.array(vars(ms)['samples']['chunk'])
avg_mem = np.mean(array_data[:, 1])/1e9
max_mem = np.max(array_data[:, 1])/1e9
client.shutdown()
cluster.close()
logging.warning('Avg memory used: %.2f GiB, Peak memory used: %.2f GiB', avg_mem, max_mem)
ave_result.plot()
```